In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import math
from scripts.config import Config
from scripts.model import Model

In [ ]:
def poly_area(x, y):
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))


def group_area(group, layout):
    coordinates = [layout.get(node_id) for node_id in group]
    south = min(coordinates, key=lambda x: x[1])
    west = min(coordinates, key=lambda x: x[0])
    north = max(coordinates, key=lambda x: x[1])
    east = max(coordinates, key=lambda x: x[0])
    bbox = [south, west, north, east]
    return poly_area([n[0] for n in bbox], [n[1] for n in bbox])

In [ ]:
config = Config()
model = Model(config)
dol, all_nodes = model.get_graph()

G = nx.Graph(dol)
edges = set(G.edges)
layout = {node.id: (float(node.lon), float(node.lat)) for node in all_nodes}
for node_name in G.nodes:
    G.nodes[node_name]['pos'] = layout[node_name]

sorted_groups = sorted(nx.connected_components(G), \
                       key=lambda g: group_area(g, layout), reverse=True)

In [ ]:
nx.draw_networkx(G, pos=layout, with_labels=False, node_size=5)
nx.draw_networkx(G.subgraph(list(sorted_groups[0])), pos=layout, node_color='r', \
                 edge_color='r', with_labels=False,
                 node_size=5)
nx.draw_networkx(G.subgraph(list(sorted_groups[1])), pos=layout, node_color='m', \
                 edge_color='m', with_labels=False,
                 node_size=5)

In [ ]:
new_edges = set(nx.geometric_edges(G, radius=config.neighbour_eps))
edges = edges.union(new_edges)

graph = nx.Graph(edges)
new_sorted_groups = sorted(nx.connected_components(graph), \
                           key=lambda g: group_area(g, layout), reverse=True)
nx.draw_networkx(graph, pos=layout, with_labels=False, node_size=5)
nx.draw_networkx(graph.subgraph(list(new_sorted_groups[0])), pos=layout, node_color='r', \
                 edge_color='r', with_labels=False,
                 node_size=5)
nx.draw_networkx(graph.subgraph(list(new_sorted_groups[1])), pos=layout, node_color='m', \
                 edge_color='m', with_labels=False,
                 node_size=5)

In [ ]:
config_zero_threshold = Config(threshold=0)
model_zero = Model(config_zero_threshold)
dol, all_nodes = model_zero.get_graph()

In [ ]:
H = nx.Graph(dol)
layout = {node.id: (float(node.lon), float(node.lat)) for node in all_nodes}
for node_name in H.nodes:
    H.nodes[node_name]['pos'] = layout[node_name]

sorted_groups = sorted(nx.connected_components(H), \
                       key=lambda g: group_area(g, layout), reverse=True)

In [ ]:
new_edges = set(nx.geometric_edges(H, radius=config.neighbour_eps))
edges = edges.union(new_edges)
H_close = nx.Graph(edges)

In [ ]:
new_sorted_groups = sorted(nx.connected_components(H_close), \
                       key=lambda g: group_area(g, layout), reverse=True)

In [ ]:
sorted([len(x) for x in new_sorted_groups], reverse=True)

In [ ]:
nx.draw_networkx(H, pos=layout, with_labels=False, node_size=1)
# nx.draw_networkx(H.subgraph(list(sorted_groups[0])), pos=layout, node_color='r', edge_color='r', with_labels=False,
#                  node_size=5)
# nx.draw_networkx(H.subgraph(list(sorted_groups[1])), pos=layout, node_color='m', edge_color='m', with_labels=False,
#                  node_size=5)

In [ ]:
sorted([len(x) for x in sorted_groups], reverse=True)

In [ ]:
largest, larger = sorted_groups[0], sorted_groups[1]
largest_centre = max(largest, key=lambda n: math.dist(layout[n], model.data_fetcher.get_centre()))
larger_centre = max(larger, key=lambda n: math.dist(layout[n], model.data_fetcher.get_centre()))
shortest_path = nx.shortest_path(H_close, largest_centre, larger_centre)
shortest_path_edges = [(shortest_path[i], shortest_path[i + 1]) for i in range(len(shortest_path) - 1)]

In [ ]:
shortest_path_edges

In [ ]:
new_sorted_groups = sorted(nx.connected_components(graph), \
                           key=lambda g: group_area(g, layout), reverse=True)
nx.draw_networkx(graph, pos=layout, with_labels=False, node_size=5)
nx.draw_networkx(graph.subgraph(list(new_sorted_groups[0])), pos=layout, node_color='r', \
                 edge_color='r', with_labels=False,
                 node_size=5)
nx.draw_networkx(graph.subgraph(list(new_sorted_groups[1])), pos=layout, node_color='m', \
                 edge_color='m', with_labels=False,
                 node_size=5)
nx.draw_networkx(nx.Graph(shortest_path_edges), pos=layout, node_color='y', \
                 edge_color='y', with_labels=False,
                 node_size=5)